## Participant-level analysis pipeline for calculating HRV

<span style='color: blue'>Version 1.1 (2023-07-28)<br></span>Dr. Daniel Holt (APKS) and Dr. Torsten Wüstenberg (CNSR)

**Aim:** Calculate resting HRV for ECG data from BrainVision data file.

**Gereral analysis steps:**

1. *Select and load data for analysis*
2. *Determine resting period*
3. *Data overview*
4. *Correct residual artifacts*
5. *Calculate and save HRV analysis*


## Preparations 

### Import libraries

The analysis pipeline is based on the following libraries. In case of an error in the execution of this cell, probably one or more of the libraries is not installed. In this case, start a terminal in **ANACONDA.NAVIGATOR** *Environments>Terminal* and install the library in question using the command </br><span style='color: red'>*pip install [library name]*</span>.

In [ ]:
import mne
import neurokit2 as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipyfilechooser

mne.set_log_level("ERROR")  # only show errors (no warnings or information messages)
results_csv = "HRV_Results.csv"  # save analysis results to this file

## Pipeline

### 1) Select and load data for analysis

Run the cell below and click "Select" to choose the BrainVision **.vhdr** file of the data set to be analyzed. The participant ID defaults to the file name, change the line `participant = ...` for setting a different value.

In [ ]:
fc = ipyfilechooser.FileChooser(".")
display(fc)

In [ ]:
%matplotlib qt
participant = fc.selected_filename[-8:-5]
raw = mne.io.read_raw_brainvision(fc.selected, preload=True)
raw.pick(["ECG"])
raw.filter(picks=["ECG"], l_freq=1, h_freq=None)
raw.plot(scalings=2e-3, duration=30)

### 1) Determine resting period

Resting period starts at marker "Stimulus S/ 10" and lasts for eight minutes (480 secs).

In [ ]:
start_event, _ = mne.events_from_annotations(raw, regexp=".*10$")
start_time = start_event[0, 0] / 500  # 500 Hz sampling rate
end_time = start_time + 480
raw.crop(start_time - 0.1, end_time)  # start 0.1 sec earlier to include first marker
raw.plot(scalings=2e-3, duration=30)

### 2) Data overview

General overview of data and peak detection. Screen for (uncorrected) artifacts and unusual data.

In [ ]:
ecg_raw = raw.get_data("ECG")[0]
ecg_preclean = nk.ecg_clean(ecg_raw, sampling_rate=500)
signals, info = nk.ecg_process(ecg_preclean, sampling_rate=500)
peaks = info["ECG_R_Peaks"]

peaks_signal, info = nk.ecg_peaks(
    ecg_preclean, sampling_rate=500, correct_artifacts=True
)
nk.hrv(peaks_signal, sampling_rate=500, show=True)
nk.ecg_plot(signals, sampling_rate=500, show_type="default")

### 3) Correct residual artifacts

Show unusual IBIs/successive differences not fixed by automatic artifact correction for closer inspection (screening criterion: 3 SDs). Drop from analysis or set to interpolated value.

In [ ]:
ibi = np.diff(peaks) * 2  # 500 Hz sampling rate
sdif = np.diff(ibi)

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(10)
n, bins, patches = ax1.hist(ibi, np.arange(500, 1501, 50))
ax1.set_title("Inter-beat intervals")
ax1.vlines(
    ibi.mean() + [-3 * ibi.std(), 3 * ibi.std()],
    *ax1.get_ylim(),
    colors="red",
    linestyles="dashed",
)
ax2.hist(sdif)
ax2.set_title("Successive differences")
ax2.vlines(
    [-3 * sdif.std(), 3 * sdif.std()],
    *ax2.get_ylim(),
    colors="red",
    linestyles="dashed",
)

In [ ]:
unusual_sdifs = np.where(abs(sdif) > 3 * sdif.std())[0]
# mark peak samples from which unusual sdifs are calculated
mark_peaks = np.repeat(unusual_sdifs, 3) + np.tile([0, 1, 2], len(unusual_sdifs))
events = np.array(
    list(zip(peaks[mark_peaks] - 10, [0] * 3 * len(unusual_sdifs), mark_peaks))
)
raw.set_annotations(mne.annotations_from_events(events, 500))
raw.plot(scalings=2e-3, duration=30)

In [ ]:
exclude_input = input(
    "Peaks von Analyse ausschliessen (Peak-Index durch Kommata getrennt): "
)

In [ ]:
exclude_ibi = [int(sample) - 1 for sample in exclude_input.split()]
ibi_del = np.delete(ibi, exclude_ibi)
peaks_adj = nk.intervals_to_peaks(ibi_del)

### 3) Calculate and save HRV indices

Calculate several HRV indeces for the resting perios using NeuroKit. Scr

In [ ]:
hrv_analysis = nk.hrv(peaks_adj, show=True)[
    ["HRV_MeanNN", "HRV_RMSSD", "HRV_HF", "HRV_LF"]
].round(5)
hrv_analysis.insert(0, "PID", participant)
results = pd.read_csv(results_csv)
results = pd.concat([results, hrv_analysis])
results.to_csv(results_csv, index=False)
print(f'Analyse-Ergebnis in "{results_csv}" gespeichert.')
results